In [16]:
from pyspark import SparkConf,SparkContext
conf=SparkConf().setAppName('Filter-Demo')
sc = SparkContext.getOrCreate(conf=conf)

In [17]:
##Creating Nested Lists of Web Pages with Outbound Links
pageLinks = [['a', ['b','c','d']],
['c', ['b']],['b', ['d','c']],['d', ['a','c']]]
 
##Initializing Rank #1 to all the webpages
pageRanks = [['a',1],['c',1],['b',1],['d',1]]

In [18]:
##Defining the number of iterations for running the page rank
###It will return the contribution to the page rank for the list of URIs
def rankContribution(uris, rank):
    numberOfUris = len(uris)
    rankContribution = float(rank) / numberOfUris
    newrank =[]
    for uri in uris:
        newrank.append((uri, rankContribution))
        return newrank

In [19]:
##Creating paired RDDs of link data
pageLinksRDD = sc.parallelize(pageLinks, 2)
pageLinksRDD.collect()

[['a', ['b', 'c', 'd']], ['c', ['b']], ['b', ['d', 'c']], ['d', ['a', 'c']]]

In [20]:
##Creating the paired RDD of our rank data 
pageRanksRDD = sc.parallelize(pageRanks, 2)
pageRanksRDD.collect()

[['a', 1], ['c', 1], ['b', 1], ['d', 1]]

In [21]:
##Defining the number of iterations and the damping factor, s
numIter = 20
s = 0.85
 
##Creating a Loop for Updating Page Rank
for i in range(numIter):
    linksRank = pageLinksRDD.join(pageRanksRDD)
    contributedRDD = linksRank.flatMap(lambda x : rankContribution(x[1][0],x[1][1]))
    sumRanks = contributedRDD.reduceByKey(lambda v1,v2 : v1+v2)
    pageRanksRDD = sumRanks.map(lambda x : (x[0],(1-s)+s*x[1]))
 
pageRanksRDD.collect()

[('a', 0.2538342489048003),
 ('b', 0.22191970477581138),
 ('d', 0.2443158819394154)]